<a href="https://colab.research.google.com/github/ArshadP443/Rotating-Screw-CNN/blob/main/mitesobjectdetectiontest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Importing Useful Libraries**


In [ ]:
import os
import re
from time import time

import numpy as np
import pandas as pd

import torch

import torch.nn as nn
from torch.nn import functional as F

from torch.utils.data import Dataset, DataLoader

In [ ]:
# For Reproduction
torch.manual_seed(2025)
torch.cuda.manual_seed(2025)

In [ ]:
#A Colab pro environment should have >20Gb of total memory.
from psutil import virtual_memory
colab_pro = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(colab_pro))

if colab_pro < 20:
  print('Not using a high-RAM runtime')
  # train model with lower settings
else:
  print('You are using a high-RAM runtime!')
  # train model with higher settings

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


#**Creating the Dataset and DataLoader**

###Mounting the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Creating the Data Frame

####Loading the Annotations


In [ ]:
labels_df = pd.read_csv(r'/content/drive/MyDrive/Object Detection/Annotations.csv')
labels_df = labels_df.fillna(-1)
labels_df = labels_df.astype(int)

####Creating a Function to Check a Frame's Label

In [ ]:
def IsPositive(video_number, frame_number):
  return int(frame_number in labels_df[f'Video {video_number}'].values)

####Creating the Main Dataset Dataframe

In [ ]:
os.chdir(IMAGE_TENSORS_DIR)
for i, pt_file_name in enumerate(os.listdir(IMAGE_TENSORS_DIR)):

  if i < 43:
    continue

  elif i < 44:

    print(pt_file_name)
    start_time = time()

    video_number = int(re.search(r'\d+', pt_file_name).group())
    video_frames = os.listdir(f'/content/drive/MyDrive/Object Detection/Videos/video{video_number}/')

    for j, image_tensor in enumerate(torch.load(pt_file_name)):
      data_df = data_df._append({'Video Number' : video_number,
                                 'Frame Number': int(re.search('frame\d+', video_frames[j]).group().replace('frame', '')),
                                 'Tensor': image_tensor,
                                 'Label': IsPositive(video_number, int(re.search('frame\d+', video_frames[j]).group().replace('frame', '')))}, ignore_index=True)

    print(f'Finished {pt_file_name} in {round(time() - start_time, 2)} seconds')

  else:
    break



video43.pt
Finished video43.pt in 23.97 seconds


In [ ]:
torch.save(data_df, 'df.pt')

###Loading the Dataframe

In [ ]:
IMAGE_TENSORS_DIR = '/content/drive/MyDrive/Object Detection/ImageTensors'

In [ ]:
os.chdir('/content/drive/MyDrive/Object Detection/TrainTensors')
data_df = torch.load('df.pt')

In [ ]:
print(data_df.shape)
print(data_df.head())

(13507, 4)
  Video Number Frame Number  \
0           17          443   
1           17          411   
2           17          478   
3           17          413   
4           17          437   

                                              Tensor  Label  
0  [[[tensor(0.6706), tensor(0.6745), tensor(0.67...  False  
1  [[[tensor(0.6706), tensor(0.6745), tensor(0.67...  False  
2  [[[tensor(0.6745), tensor(0.6745), tensor(0.67...  False  
3  [[[tensor(0.6706), tensor(0.6745), tensor(0.67...  False  
4  [[[tensor(0.6745), tensor(0.6784), tensor(0.67...  False  


In [ ]:
data_df['Label'].value_counts()

Label
False    13090
True       417
Name: count, dtype: int64

In [ ]:
os.chdir(IMAGE_TENSORS_DIR)
for i, pt_file_name in enumerate(os.listdir(IMAGE_TENSORS_DIR)):

  if i < 43:
    continue

  elif i < 44:

    print(pt_file_name)
    start_time = time()

    video_number = int(re.search(r'\d+', pt_file_name).group())
    video_frames = os.listdir(f'/content/drive/MyDrive/Object Detection/Videos/video{video_number}/')

    for j, image_tensor in enumerate(torch.load(pt_file_name)):
      data_df = data_df._append({'Video Number' : video_number,
                                 'Frame Number': int(re.search('frame\d+', video_frames[j]).group().replace('frame', '')),
                                 'Tensor': image_tensor,
                                 'Label': IsPositive(video_number, int(re.search('frame\d+', video_frames[j]).group().replace('frame', '')))}, ignore_index=True)

    print(f'Finished {pt_file_name} in {round(time() - start_time, 2)} seconds')

  else:
    break

In [ ]:
# This is just a quick fix for a problem when creating the dataframel; pls fix when making a new dataframe
data_df['Label'] = data_df['Label'].astype(int)

In [ ]:
data_df_test = pd.DataFrame(columns=['Video Number', 'Frame Number', 'Tensor', 'Label'])

In [ ]:
os.chdir(IMAGE_TENSORS_DIR)
for i, pt_file_name in enumerate(os.listdir(IMAGE_TENSORS_DIR)):

  if i < 44:
    continue

  elif i < 49:

    print(pt_file_name)
    start_time = time()

    video_number = int(re.search(r'\d+', pt_file_name).group())
    video_frames = os.listdir(f'/content/drive/MyDrive/Object Detection/Videos/video{video_number}/')

    for j, image_tensor in enumerate(torch.load(pt_file_name)):
      data_df_test = data_df_test._append({'Video Number' : video_number,
                                 'Frame Number': int(re.search('frame\d+', video_frames[j]).group().replace('frame', '')),
                                 'Tensor': image_tensor,
                                 'Label': IsPositive(video_number, int(re.search('frame\d+', video_frames[j]).group().replace('frame', '')))}, ignore_index=True)

    print(f'Finished {pt_file_name} in {round(time() - start_time, 2)} seconds')

  else:
    break

video44.pt
Finished video44.pt in 4.36 seconds
video46.pt
Finished video46.pt in 13.42 seconds
video47.pt
Finished video47.pt in 18.93 seconds
video48.pt
Finished video48.pt in 15.33 seconds
video49.pt
Finished video49.pt in 18.04 seconds


In [ ]:
data_df_test['Label'].value_counts()

Label
0    6058
1     207
Name: count, dtype: int64

In [ ]:
os.chdir('/content/drive/MyDrive/Object Detection/TestTensors')
torch.save(data_df_test, 'df_test.pt')

###Creating the Dataset

In [ ]:
class FramesDataset(Dataset):
  def __init__(self, data_pd):
    self.data_pd = data_pd

  def __len__(self):
    return len(self.data_pd)

  def __getitem__(self, index):
    return self.data_pd.iloc[index]['Tensor'], self.data_pd.iloc[index]['Label']

###Creating the DataLoader

In [ ]:
train_dataset = FramesDataset(data_df)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)

#**The Neural Network**

##Creating the Neural Network

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, 5, 2)
        self.conv2 = nn.Conv2d(4, 16, 5, 2)
        self.fc1 = nn.Linear(576, 128)
        self.fc2 = nn.Linear(128, 8)
        self.fc3 = nn.Linear(8, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 3, 3)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 3, 3)
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

##Training the Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

In [ ]:
# Hyperparameters
epochs = 4
learning_rate = .001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {param_count:,} trainable parameters')

The model has 76,617 trainable parameters


In [ ]:
model = torch.compile(model, mode="reduce-overhead")
torch.set_float32_matmul_precision("medium")
torch.backends.cudnn.benchmark = True

In [ ]:
for batch_idx, (data, targets) in enumerate(train_dataloader):
    with torch.autocast(device_type="cuda"):
        outputs = model(data.to(device))
        loss = criterion(outputs, targets.to(device))
    print(loss)
    break

In [ ]:
times = []
losses = []

for epoch in range(epochs):
    for batch_idx, (data, targets) in enumerate(train_dataloader):

        start = time()

        data = data.to(device=device)
        targets = targets.to(device=device)


        outputs = model(data.to(device))
        loss = criterion(outputs, targets.to(device))
        for param in model.parameters():
            param.grad = None

        loss.backward()
        optimizer.step()


        losses.append(loss.item())
        times.append(time() - start)

        msg =  f'Epoch: {epoch+1:02}/{epochs:02} | '
        msg += f'Batch: {batch_idx+1:05}/{len(train_dataloader):05} | '
        msg += f'Loss: {losses[-1]:0.4} | '
        msg += f'Avg Loss: {sum(losses)/len(losses):0.4} | '
        msg += f'Time: {times[-1]:0.4} | '
        avg_time = sum(times)/len(times)
        msg += f'Avg Time: {avg_time:0.4} | '
        msg += f'Time Left: {avg_time * (len(train_dataloader) - (batch_idx + 1)):0.4}'
        print(msg, flush=True)
        if (batch_idx%10 == 0):
            torch.cuda.empty_cache()

In [ ]:
transform = Compose((Resize((256, 256)), Grayscale(), ToTensor()))
image = transform(Image.open('/content/drive/MyDrive/Object Detection/Videos/video46/frame173.jpeg'))

image = image.view(1, 1, 256, 256)

torch.Size([1, 1, 256, 256])

In [ ]:
with torch.no_grad():
  y_pred = model(image.view(1, 1, 256, 256).to(device))
  print(y_pred)

tensor([[ 12.5323, -11.9458]], device='cuda:0')


In [ ]:
print(label)

0


#**Converting the Images to Tensors**


###Importing Useful Libraries

In [ ]:
import os
import gc
from time import time

import torch
from torchvision.transforms import Compose, ToTensor, Resize, Grayscale

from PIL import Image

###Mount the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Getting the File Paths for EVERY Frame in a Given Video

In [ ]:
VIDEO_NUMBERS = (48, 49)

# The file directories for each video
video_directories = [f'/content/drive/MyDrive/Object Detection/Videos/video{video_number}' for video_number in VIDEO_NUMBERS]

# List containing the file paths for every frame in specified videos
frame_directories = [[os.path.join(video_directory, frame) for frame in os.listdir(video_directory)] for video_directory in video_directories]

###Converting & Storing Image Tensors Into a List

In [ ]:
os.chdir(r'/content/drive/MyDrive/Object Detection/ImageTensors')
transform = Compose((Resize((256, 256)), Grayscale(), ToTensor()))

for i in range(len(VIDEO_NUMBERS)):
  start_time = time()

  image_tensors = [transform(Image.open(path)) for path in frame_directories[i]]

  torch.save(image_tensors, f'video{VIDEO_NUMBERS[i]}.pt')
  print(f'Saved video #{VIDEO_NUMBERS[i]} in {round((start_time - time()) / 60, 2)} minutes')

  gc.collect()

print('Done!')

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/Object Detection/Videos/video40')))

1349


In [ ]:
os.chdir(r'/content/drive/MyDrive/Object Detection/ImageTensors')
transform = Compose((Resize((256, 256)), Grayscale(), ToTensor()))


start_time = time()
image_tensors = []

videodir = '/content/drive/MyDrive/Object Detection/Videos/video49'
for frame in os.listdir(videodir):
  path = os.path.join(videodir, frame)
  tensor = transform(Image.open(path))
  image_tensors.append(tensor)

torch.save(image_tensors, 'video49.pt')
print(f'Saved video #49 in {round((start_time - time()) / 60, 2)} minutes')

gc.collect()

print('Done!')

Saved video #49 in -1.32 minutes
Done!
